In [ ]:
# 温度、湿度、温湿度都绘制
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Times New Roman']  # 指定默认
plt.rcParams['axes.unicode_minus'] = False
# ===== 配置 =====

In [ ]:
# -----------------------------
# 1. 导入包
# -----------------------------
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
plt.rcParams['font.sans-serif'] = ['Times New Roman']  # 指定默认
plt.rcParams['axes.unicode_minus'] = False
# -----------------------------
# 2. 读取数据
# -----------------------------
data = pd.read_excel("D:/Permafrost/temp/var/varr.xlsx", index_col=0)
#标准化到0-1
data = pd.DataFrame(MinMaxScaler().fit_transform(data), columns=data.columns)
# 选择数值变量
num_data = data.select_dtypes(include=["number"])

# -----------------------------
# 3. 计算 Pearson 相关矩阵与显著性
# -----------------------------
corr = num_data.corr(method="pearson")
p_values = pd.DataFrame(np.ones_like(corr), columns=corr.columns, index=corr.index)

for i in corr.columns:
    for j in corr.columns:
        if i != j:
            r, p = pearsonr(num_data[i], num_data[j])
            corr.loc[i, j] = r
            p_values.loc[i, j] = p
# -----------------------------
# 4. 绘制仅下三角散点图矩阵 + 拟合曲线
# -----------------------------
sns.set(style="white", context="talk")
g = sns.PairGrid(num_data)
# 在下三角绘制散点和线性拟合
def scatter_with_fit(x, y, **kwargs):
    sns.regplot(x=x, y=y, scatter_kws={"s": 40, "edgecolor": "#06153ED2", "alpha": 0.7}, line_kws={"color": "red"}, **kwargs)
g.map_lower(scatter_with_fit)
# 上三角和对角线留空
for i in range(len(g.axes)):
    for j in range(len(g.axes)):
        if i <= j:
            g.axes[i, j].set_visible(False)
#设置字体大小
label_fontsize = 40   # 轴标签字体大小
tick_fontsize = 25    # 刻度字体大小
for ax_row in g.axes:
    for ax in ax_row:
        if ax is not None:
            ax.set_xlabel(ax.get_xlabel(), fontsize=label_fontsize)
            ax.set_ylabel(ax.get_ylabel(), fontsize=label_fontsize)
            ax.tick_params(axis='both', labelsize=tick_fontsize)
plt.show()


In [ ]:
# -----------------------------
from matplotlib.collections import LineCollection
from matplotlib.colors import BoundaryNorm, ListedColormap
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

plt.close('all')
fig, ax = plt.subplots(figsize=(10, 8))

# -----------------------------
# 掩膜：只显示上三角
# -----------------------------
mask = np.zeros_like(corr, dtype=bool)
# np.tril_indices_from(mask) 下三角
mask[np.tril_indices_from(mask)] = True   ## 下三角遮住
mask[np.diag_indices_from(mask)] = True

# -----------------------------
# 自定义颜色带（与你提供的相同）
# -----------------------------
bounds = [-1, -0.7, -0.3, -0.1, 0.1, 0.3, 0.7, 1.0]
colors = ["#A53737", "#B55959", "#DABEBE", "#CED6DB",
          "#0D7DCD", "#1E5E8C", "#15405F"]
cmap = ListedColormap(colors)
norm = BoundaryNorm(bounds, cmap.N)

# -----------------------------
# 绘制空白背景（不显示色块）
# -----------------------------
sns.heatmap(
    corr,
    mask=mask,
    cmap=[[1, 1, 1, 0]],  # 用透明背景代替色块
    cbar=False,
    linewidths=0, 
    ax=ax
)

# -----------------------------
# 改为绘制圆点，大小随值变化
# -----------------------------
n = corr.shape[0]
for i in range(n):
    for j in range(n):
        if not mask[i, j] and not np.isnan(corr.iat[i, j]):
            v = corr.iat[i, j]
            color = cmap(norm(v))             # 按自定义色带取色
            size = abs(v) * 1000             # ✅ 圆点大小可调（1800 可以改成更大/更小）
            ax.scatter(j + 0.5, i + 0.5,
                       s=size,
                       color=color,
                       edgecolor='k',
                       linewidth=0.3,
                       alpha=1)

# -----------------------------
# 坐标轴与色带
# -----------------------------
top,bt, lf,rt = True,False, False,True
ax.tick_params(axis='x', which='both',
               top=top, labeltop=top,
               bottom=bt, labelbottom=bt)
ax.tick_params(axis='y', which='both',
               left=lf, labelleft=lf,
               right=rt, labelright=rt)
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha='left', fontsize=15)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=15)

# 添加独立色带（与原 norm & cmap 对应）
sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
cbar = plt.colorbar(sm, ax=ax, shrink=0.8, pad=0.08, ticks=bounds)
cbar.ax.tick_params(labelsize=12)
cbar.set_label("Pearson r", fontsize=13)

# plt.savefig("./pic/heatmap_bubble.png", dpi=900, bbox_inches='tight', transparent=True)
plt.show()
